In [1]:
#Loading Dependencies
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from os import listdir
from os.path import isfile, join

In [2]:
#Setting the CSV variables
population_census = "Resources/population_by_zip_2010.csv"
restaurant_data = "Resources/FastFoodRestaurants.csv"

In [3]:
#Loading CSV into DataFrames
populationDF = pd.read_csv(population_census, encoding="utf-8")
restaurantDF = pd.read_csv(restaurant_data, encoding="utf-8")

In [77]:
#Cleaning DFs, taking only the columns we need
cleaned_population = populationDF[['zipcode','population']].copy()
cleaned_restaurant = restaurantDF[['city','name','postalCode','province']].copy()

#Renaming columns
cleaned_restaurant.rename(columns={'postalCode':'zipcode'}, inplace=True)

#Changing DataType to match, INT --> OBJECT
#NOTE; restaurant DF can not be changed to INT due to some zipcodes having a '-'. 
#NOTE; Because of this '-' we must change the datatype to an OBJECT instead, treat as a text
cleaned_population['zipcode'] = cleaned_population['zipcode'].astype(object)

#Setting zipcode as index
#NOTE, setting the index for zip code might NOT be necessary; experiementing
#NOTE, renaming the zipcode column in both DFs to a singular name may be beneficial; experimenting
#CODE;cleaned_more_pop.set_index('zipcode', inplace=True)
#CODE;cleaned_restaurant.set_index('zipcode', inplace=True)
cleaned_restaurant

#NEXTLINE; finding a way to merge DFs using zip code as a base and inserting population into the new DF

,city,name,zipcode,province
0,Massena,McDonald's,13662,NY
1,Washington Court House,Wendy's,43160,OH
2,Maysville,Frisch's Big Boy,41056,KY
3,Massena,McDonald's,13662,NY
4,Athens,OMG! Rotisserie,45701,OH
5,Hamilton,Domino's Pizza,45011,OH
6,Englewood,Domino's Pizza,45322,OH
7,Saluda,McDonald's,29138,SC
8,Batesburg,Wendy's,29006,SC
9,Paragould,Pizza Hut,72450,AR


In [66]:
cleaned_population.head()

,zipcode,population
0,99776,124
1,38305,49808
2,37086,31513
3,41667,720
4,67001,1676


In [65]:
cleaned_restaurant.head()

,city,name,zipcode,province
0,Massena,McDonald's,13662,NY
1,Washington Court House,Wendy's,43160,OH
2,Maysville,Frisch's Big Boy,41056,KY
3,Massena,McDonald's,13662,NY
4,Athens,OMG! Rotisserie,45701,OH
